In [ ]:
# default_exp gh

# gh

> Tools for GitHub releases

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import re
import yaml
import pkgutil
import subprocess
from configparser import ConfigParser

In [ ]:
#export
def write_gh_template():
    "Write GitHub template for assigning commit messages to categories"
    template = pkgutil.get_data(__name__, "templates/gh_release.yaml").decode("utf-8")
    with open(".gh-release-config.yaml", "w") as f:
        f.write(template)

In [ ]:
#export
def get_last_tag():
    "Get latest git tag"
    return subprocess.run(["git", "describe", "--tags", "--abbrev=0"], 
                          capture_output=True).stdout.decode("utf").strip()

In [ ]:
#export
def get_commit_msgs(from_tag=None, to="HEAD"):
    "Get commits `from_tag` to `to`. If `from_tag` is None then it will get the commits from latest tag"
    last_tag = from_tag if from_tag is not None else get_last_tag()
    commit_msgs = subprocess.run(["git", "log", f"{last_tag}..{to}", "--pretty=%s"],
                          capture_output=True).stdout.decode("utf").strip()
    return commit_msgs.split("\n")

In [ ]:
#export
def get_config():
    "Get config from settings.ini"
    config = ConfigParser(delimiters=['='])
    config.read('settings.ini')
    cfg = config['DEFAULT']
    return cfg

In [ ]:
#export
def get_release_version():
    "Get release version from settings.ini"
    cfg = get_config()
    return cfg["version"]

In [ ]:
#export
def add_pull_request_link(msg):
    "Add pull request link to commit message `s` if reference is found, e.g. #100"
    cfg = get_config()
    git_url = cfg["git_url"]
    msg_with_link = re.sub("(#)(\d+)", rf"[\1\2]({git_url}/pull/\2)", msg)
    return msg_with_link

In [ ]:
#export
def release_report(report_title="", from_tag=None, to="HEAD"):
    "Release report `from_tag` to commit `to` using `report_title`"
    commit_msgs = get_commit_msgs(from_tag, to)
    with open(".gh-release-config.yaml") as f:
        gh_config = yaml.safe_load(f)
    gh_categories = gh_config["gh_categories"]
    report_list = [report_title]
    for d in gh_categories:
        gh_cat_commits = [add_pull_request_link(f"* {msg}") for msg in commit_msgs if d["keyword"] in msg]
        gh_cat_report = f"{d['title']}\n" + "{}".format("\n".join(gh_cat_commits)) if len(gh_cat_commits) > 0 else ""
        report_list.append(gh_cat_report)
    report_list = [rep for rep in report_list if rep != ""]
    report = "\n\n".join(report_list)
    return report

In [ ]:
#export
def get_token():
    "Get GitHub token from file .gh_token"
    with open(".gh-token", "r") as f:
        token = f.read().strip()
    return token

In [ ]:
#export
def make_gh_release():
    "Make a GitHub release"
    # get config
    cfg = get_config()
    # get token
    token = get_token()
    # get release version
    release_version = cfg["version"]
    # get release report
    report = release_report()
    # publish release to GitHub
    # TODO
    # download new tag from GitHub
    subprocess.run(["git", "fetch"])

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_gh.ipynb.
Converted index.ipynb.
